In [ ]:
import json
from collections import Counter
from pprint import pprint
from tqdm import tqdm
import json
from copy import deepcopy
import yaml

In [ ]:
data_generated = json.load(open('eval_generations/eval_7b_testv1_train/api_data_0_468.json', 'r'))
data_ground_truth = json.load(open('ToolAlpaca/data/train_data.json'))

Посмотрим на то, какой длины цепочки в ground truth

In [ ]:
lengths = []
for item in data_ground_truth:
    for chain in item['Instances']:
        if 'intermediate_steps' in chain:
            lengths.append(len(chain['intermediate_steps']))
Counter(lengths)

In [ ]:
data_generated[0]

In [ ]:
errors = []
count = 0
for idx, (chain_pred, chain_gt) in enumerate(zip(data_generated, data_ground_truth)):
    for query, steps_pred, steps_gt in zip(chain_pred['Instructions'], 
                                           chain_pred['Instances'], 
                                           chain_gt['Instances']):
        if not 'intermediate_steps' in steps_pred:
            errors.append({
                'tool_set_num': idx,
                'type': 'other_error_pred',
                'query': query,
                'steps_pred': steps_pred,
                'steps_gt': steps_gt,
            })
            count += 1
            continue
        if not 'intermediate_steps' in steps_gt:
            errors.append({
                'tool_set_num': idx,
                'type': 'other_error_gt',
                'query': query,
                'steps_pred': steps_pred,
                'steps_gt': steps_gt,
            })
            count += 1
            continue
        if len(steps_pred['intermediate_steps']) == 0 or len(steps_gt['intermediate_steps']) == 0:
            count += 1
            continue
        steps_pred = steps_pred['intermediate_steps'][0][0]
        steps_gt = steps_gt['intermediate_steps'][0][0]
        if steps_pred[0] != steps_gt[0]:
            errors.append({
                'tool_set_num': idx,
                'type': 'name_error',
                'query': query,
                'steps_pred': steps_pred,
                'steps_gt': steps_gt,
            })
            count += 1
        else: 
            try: 
                params_pred = json.loads(steps_pred[1])
                params_gt = json.loads(steps_gt[1])
                if params_gt != params_pred:
                    errors.append({
                        'tool_set_num': idx,
                        'type': 'parameter_error',
                        'query': query,
                        'steps_pred': steps_pred,
                        'steps_gt': steps_gt,
                    })
                    count += 1
            except json.decoder.JSONDecodeError as err:
                errors.append({
                    'tool_set_num': idx,
                    'type': 'parameter_error',
                    'query': query,
                    'steps_pred': steps_pred,
                    'steps_gt': steps_gt,
                })
                count += 1

In [ ]:
len(errors)

In [ ]:
errors

In [ ]:
import sys
sys.path.append('ToolAlpaca')
from utils import load_openapi_spec, analyze_openapi_spec
import json
from agent.convert_request import call_api_function
from utils import load_openapi_spec, escape
from agent.tools import Tool, GetDetailsTool, tool_projection

def load_tools(api_data):
    server_url = "http://127.0.0.1:5678"
    openapi_spec = load_openapi_spec(api_data["Documentation"], replace_refs=True)
    components_descriptions = escape(api_data["Function_Description"]["components"])
    tools = dict()
    for idx, func_name in enumerate(api_data["Function_Projection"]):
        description = escape(api_data["Function_Description"][func_name])
        if idx == len(api_data["Function_Projection"]) - 1:
            description += components_descriptions
        path, method = api_data["Function_Projection"][func_name]
        tools[func_name] = Tool(
            base_url=server_url + "/" + api_data["Name"] if server_url else None,
            func_name=func_name,
            openapi_spec=openapi_spec,
            path=path,
            method=method,
            description=description,
            retrieval_available="retrieval" in api_data.get("external_tools", [])
        )
    return tools

In [ ]:
count = 0
for item in tqdm(errors):
    if not 'error' in item['steps_gt'] and not 'error' in item['steps_pred'] and item['type'] == 'name_error':
        tool_set = load_tools(data_ground_truth[item['tool_set_num']])
        tool_name_pred = item['steps_pred'][0]
        tool_params_pred = item['steps_pred'][1]    
        tool_res_pred = tool_set[tool_name_pred](tool_params_pred)
        tool_name_gt = item['steps_gt'][0]
        tool_params_gt = item['steps_gt'][1]
        tool_res_gt = tool_set[tool_name_gt](tool_params_gt)
        print(tool_name_pred)
        print(tool_params_pred)
        print(tool_res_pred)
        print(tool_name_gt)
        print(tool_params_gt)
        print(tool_res_gt)
        print('#' * 100)

In [ ]:
count

In [ ]:
print(tool_params_pred)
print(tool_res_pred)

In [ ]:
print(tool_params_gt)
print(tool_res_gt)

In [ ]:
tool_set[tool_name_pred]

In [ ]:
tool_set

Проверим, есть ли примеры с error в поле входа

In [ ]:
errors_halluc = 0
for item in tqdm(errors):
    if 'error' in item['steps_pred']:
        pprint.pprint(item)
        print('#' * 100)
        errors_halluc += 1

Сколько всего ошибок с "other error"

In [ ]:
other_errors_gt = 0
for item in errors:
    if item['type'] == 'other_error_gt':
        pprint(item)
        other_errors_gt += 1

other_errors_gt

In [ ]:
other_errors_pred = 0
for item in errors:
    if item['type'] == 'other_error_pred':
        pprint(item)
        other_errors_pred += 1

other_errors_pred

Посмотрим, есть ли примеры, в которых в prediction просто нет нужного тула

In [ ]:
count_hallucination_names = 0
for item in tqdm(errors):
    if item['type'] != 'other_error_pred' and item['type'] != 'other_error_gt':
        tool_set = data_ground_truth[item['tool_set_num']]['Function_Description']
        pred_tool_name = item['steps_pred'][0]
        gt_tool_name = item['steps_gt'][0]
        if pred_tool_name not in tool_set:
            pprint(tool_set)
            pprint(item)
            count_hallucination_names += 1

In [ ]:
count_hallucination_names

Посмотрим, есть ли примеры, где модель использовала тул, которого нет в промпте

In [ ]:
count_not_in_tool_set = 0
for idx, chain_pred in enumerate(data_generated):
    tool_set = chain_pred['Function_Description']
    for query, steps_pred in zip(chain_pred['Instructions'], chain_pred['Instances']):                           
        if not 'intermediate_steps' in steps_pred:
            continue
        if len(steps_pred['intermediate_steps']) == 0:
            continue
        steps_pred = steps_pred['intermediate_steps'][0][0]
        if steps_pred[0] not in tool_set:
            pprint(steps_pred)
            count_not_in_tool_set += 1

In [ ]:
count_not_in_tool_set

Попробуем распарсить параметры тулов

In [ ]:
all_tool_names_params_parseble = dict()
all_tools_count = 0
for idx, chain_pred in enumerate(data_generated):
    tool_set = chain_pred['Function_Description']
    for key, val in tool_set.items():
        all_tools_count += 1
        if val:
            try:
                print(key)
                print(val)
                tool_params = json.loads(val.split('Parameters: ')[1].split('\nOutput:')[0])
                print(tool_params)
                print('#' * 100)

                all_tool_names_params_parseble[key] = tool_params
            except Exception as e:
                continue

In [ ]:
all_tool_names_params_parseble

In [ ]:
len(all_tool_names_params_parseble)

In [ ]:
all_tools_count

Теперь попробуем пройтись по всем вызовам тулов с ошибками и посмотреть на примеры, в которых нету каких-то параметров 

In [ ]:
error_count_wrong_param_names = 0
error_count_no_required_params = 0
count_correct_gt_uncorrect_pred = 0
for item in tqdm(errors):
    if item['type'] != 'other_error_pred' and item['type'] != 'other_error_gt':
        pred_tool_name = item['steps_pred'][0]
        pred_tool_params = item['steps_pred'][1]
        gt_tool_name = item['steps_gt'][0]
        gt_tool_params = item['steps_gt'][1]
        res = check_wrong_or_required_params(pred_tool_name, pred_tool_params)
        if not res[0]:
            if res[1] == 'wrong':
                error_count_wrong_param_names += 1
            else:
                error_count_no_required_params += 1
        res_gt = check_wrong_or_required_params(gt_tool_name, gt_tool_params)
        print(item)
        print(res_gt)
        if res_gt[0] and not res[0]:
            count_correct_gt_uncorrect_pred += 1
            pprint(item)

In [ ]:
error_count_wrong_param_names

In [ ]:
error_count_no_required_params

In [ ]:
count_correct_gt_uncorrect_pred

Теперь попробуем вывести все примеры, в которых в ground_truth нет ошибок, а в prediction есть

In [ ]:
def check_wrong_or_required_params(tool_name, tool_params):
    if tool_name in all_tool_names_params_parseble:
            tool_param_descriptions = all_tool_names_params_parseble[tool_name]
            for param in tool_params:
                if param not in tool_param_descriptions:
                    return (False, 'wrong')
            for param in tool_param_descriptions:
                if 'required' in tool_param_descriptions[param].lower():
                    if not param in tool_params:
                        return (False, 'no_required')

    return (True, '')

Теперь попробуем почистить трейн и выделить все примеры, в которых есть корректный вызов тулов

In [ ]:
all_train = []
cleaned_train = []

for item in data_ground_truth:
    for steps in item['Instances']:
        flag = True
        try:
            for item in steps['intermediate_steps']:
                gt_tool_name = item[0][0]
                gt_tool_params = json.loads(item[0][1])
                if not check_wrong_or_required_params(gt_tool_name, gt_tool_params)[0]:
                    flag = False
        except:
            flag = False
        if flag:
            cleaned_train.append(steps)
        all_train.append(steps)

In [ ]:
len(cleaned_train)

In [ ]:
len(all_train)

In [ ]:
item

In [ ]:
item

Попробуем почистить трейн и погенерировать ошибки на очищенном трейне

In [ ]:
errors[0]

Сколько данных в оригинальном трейне

In [ ]:
train_data_preprocessed = json.load(open('data/train_data_preprocessed', 'r'))
len(train_data_preprocessed)

Посмотрим, использует ли модель вообще в тесте тул getDetails

In [ ]:
test_generations = json.load(open('eval_generations/eval_7b_testv1_simulated/api_data_0_10.json', 'r'))

In [ ]:
all_tool_chains = 0
tool_chains_w_getdetails = 0

for item in test_generations:
    for steps in item['Instances']:
        if 'intermediate_steps' in steps:
            for item in steps['intermediate_steps']:
                gt_tool_name = item[0][0]
                if gt_tool_name == 'getDetails':
                    tool_chains_w_getdetails += 1
                    break
            all_tool_chains += 1

In [ ]:
all_tool_chains

In [ ]:
tool_chains_w_getdetails

Примеры с getDetails

In [ ]:
error_count_wrong_param_names = 0
error_count_no_required_params = 0
count_correct_gt_uncorrect_pred = 0
exception_errors = 0
for item in tqdm(errors):
    try:
        if item['type'] != 'other_error_pred' and item['type'] != 'other_error_gt':
            pred_tool_name = item['steps_pred'][0]
            pred_tool_params = json.loads(item['steps_pred'][1])
            gt_tool_name = item['steps_gt'][0]
            gt_tool_params = json.loads(item['steps_gt'][1])
            res = check_wrong_or_required_params(pred_tool_name, pred_tool_params)
            if not res[0]:
                if res[1] == 'wrong':
                    error_count_wrong_param_names += 1
                else:
                    error_count_no_required_params += 1
            res_gt = check_wrong_or_required_params(gt_tool_name, gt_tool_params)
            if res_gt[0] and not res[0]:
                count_correct_gt_uncorrect_pred += 1
                pprint(item)
    except:
        exception_errors += 1

In [ ]:
error_count_wrong_param_names

In [ ]:
error_count_no_required_params 

In [ ]:
count_correct_gt_uncorrect_pred

In [ ]:
exception_errors

In [ ]:
all_tool_names_params_parseble['getElevation']

In [ ]:
all_tool_names_params_parseble['filterByArea']

In [ ]:
all_tool_names_params_parseble['getTicker']

In [ ]:
all_tool_names_params_parseble['getAcco']

Тест функции проверки параметров

In [ ]:
def check_wrong_or_required_params(tool_name, tool_params):
    if tool_name in all_tool_names_params_parseble:
            tool_param_descriptions = all_tool_names_params_parseble[tool_name]
            for param in tool_params:
                if param not in tool_param_descriptions:
                    return (False, 'wrong')
            for param in tool_param_descriptions:
                if 'required' in tool_param_descriptions[param].lower():
                    if not param in tool_params:
                        return (False, 'no_required')
    return (True, '')

In [ ]:
check_wrong_or_required_params('getElevation', {"latitude": 34.25, "longitude": -71.02})

Генерация примеров с ошибками

In [ ]:
sample_w_error_corrections = deepcopy(data_ground_truth)

sample_w_error_corrections[0]

In [ ]:
sample_w_error_corrections

In [ ]:
steps_pred

Составим примеры с ошибками, которые мы можем детектировать с помощью неправильных параметров

In [ ]:
sample_w_error_corrections = deepcopy(data_ground_truth)

count = 0
error_count = 0
count_non_gt_non_pred = 0
count_non_gt_pred = 0
count_gt_pred = 0
count_gt_non_pred = 0
for idx, (chain_pred, chain_gt) in enumerate(zip(data_generated, data_ground_truth)):
    sample_w_error_corrections[idx]['Instructions'] = []
    sample_w_error_corrections[idx]['Instances'] = []
    
    for query_pred, query_gt, steps_pred, steps_gt in zip(chain_pred['Instructions'],
                                                          chain_gt['Instructions'],  
                                                          chain_pred['Instances'],
                                                          chain_gt['Instances']):
        
        assert query_pred == query_gt
        if ('intermediate_steps' in steps_gt and 'intermediate_steps' in steps_pred and 
            len(steps_gt['intermediate_steps']) != 0 and len(steps_pred['intermediate_steps']) != 0):
                try:
                    item_gt, item_pred = steps_gt['intermediate_steps'][0], steps_pred['intermediate_steps'][0]
                    pred_tool_name = item_pred[0][0]
                    pred_tool_params = yaml.load(item_pred[0][1], yaml.Loader)
                    gt_tool_name = item_gt[0][0]
                    gt_tool_params = yaml.load(item_gt[0][1], yaml.Loader)
                    res_pred = check_wrong_or_required_params(pred_tool_name, pred_tool_params)
                    res_gt = check_wrong_or_required_params(gt_tool_name, gt_tool_params)                                
                    if res_gt[0] and not res_pred[0]:
                        sample_w_error_corrections[idx]['Instructions'].append(query_gt)
                        item_gt_copy = deepcopy(steps_gt)
                        item_gt_copy['intermediate_steps'] = item_pred + item_gt_copy['intermediate_steps']
                        sample_w_error_corrections[idx]['Instances'].append(item_gt_copy)
                        count_gt_non_pred += 1
                    elif not res_gt[0] and not res_pred[0]:
                         count_non_gt_non_pred += 1
                    elif not res_gt[0] and res_pred[0]:
                         count_non_gt_pred += 1
                    elif res_gt[0] and res_pred[0]:
                         count_gt_pred += 1
                except Exception as e:
                    print(e)
                    error_count += 1
                    print('#' * 100)

count_correction_examples = 0
sample_w_error_corrections_final = []
for item in sample_w_error_corrections:
    if len(item['Instructions']) > 0:
        sample_w_error_corrections_final.append(item)
        count_correction_examples += len(item['Instructions'])

In [ ]:
count_gt_non_pred

In [ ]:
count_gt_pred

In [ ]:
count_non_gt_non_pred

In [ ]:
count_non_gt_pred

In [ ]:
all_tool_names_params_parseble['getEventDetails']

In [ ]:
count_correction_examples

In [ ]:
sample_w_error_corrections_final

In [ ]:
json.dump(sample_w_error_corrections_final, open('data/correction_examples_wrong_params_no_thoughts.json', 'w'), indent=4)

Почистим трейн от примеров, где нету instance, где хотя бы во одном шаге есть ошибка с параметрами

In [ ]:
cleaned_train = deepcopy(data_ground_truth)

for idx, chain_gt in enumerate(data_ground_truth):
    cleaned_train[idx]['Instructions'] = []
    cleaned_train[idx]['Instances'] = []

    for query_gt, steps_gt in zip(chain_gt['Instructions'], chain_gt['Instances']):
        if 'intermediate_steps' in steps_gt and len(steps_gt['intermediate_steps']) != 0:
            try:
                flag = True
                for item_gt in steps_gt['intermediate_steps']:
                    gt_tool_name = item_gt[0][0]
                    gt_tool_params = yaml.load(item_gt[0][1], yaml.Loader)
                    res_gt = check_wrong_or_required_params(gt_tool_name, gt_tool_params)
                    if not res_gt[0]:
                        flag = False
                if flag:
                    cleaned_train[idx]['Instructions'].append(query_gt)
                    cleaned_train[idx]['Instances'].append(steps_gt)
            except:
                continue

In [ ]:
num_dirty_train_examples = 0
for item in data_ground_truth:
    num_dirty_train_examples += len(item['Instances'])

In [ ]:
num_cleaned_train_examples = 0
for item in cleaned_train:
    num_cleaned_train_examples += len(item['Instances'])

In [ ]:
num_cleaned_train_examples

In [ ]:
num_dirty_train_examples

Сохраним эту предварительную версию трейна, чтобы на ней потом обучить

In [ ]:
json.dump(cleaned_train,  open('ToolAlpaca/data/cleaned_train_v1.json', 'w'))

Подготовим промпт и вызов ChatGPT для генерации ответов тулов с ошибками

In [ ]:
all_tool_names_params_parseble['getWeatherForecast']

In [ ]:
sample_w_error_corrections_error_from_tools = deepcopy(sample_w_error_corrections_final)

In [ ]:
for item in sample_w_error_corrections_final:
    tool_set = load_tools(item)
    for chain in item['Instances']:
        name = chain['intermediate_steps'][0][0]
        incorrect_params = chain['intermediate_steps'][0][1]
        incorrect_params = '{"location": "Marbella", "days": 7, "kek": "True"}'
        tool = tool_set[name]
        print(name, incorrect_params)
        print(tool)
        output = tool._run(incorrect_params)
        print(output)
        print('#' * 100)
        print(name, incorrect_params)
        break
    break

In [ ]:
tool_set['searchJobs']

In [ ]:
item

Скорее всего, я неправильно детектил ошибки. Пересоберу трейн с коррекшоном

In [ ]:
def check_params(tool_name, tool_params, curr_tool_descriptions_parseble, all_tool_names):
    if tool_name not in all_tool_names:
        return (False, 'wrong_name')
    if tool_name in curr_tool_descriptions_parseble:
        tool_param_descriptions = curr_tool_descriptions_parseble[tool_name]
        for param in tool_params:
            if param not in tool_param_descriptions:
                return (False, 'wrong')
        for param in tool_param_descriptions:
            if 'required' in tool_param_descriptions[param].lower():
                if not param in tool_params:
                    return (False, 'no_required')
    return (True, '')

sample_w_error_corrections = deepcopy(data_ground_truth)

count = 0
error_count = 0
count_non_gt_non_pred = 0
count_non_gt_pred = 0
count_gt_pred = 0
count_gt_non_pred = 0
for idx, (chain_pred, chain_gt) in enumerate(zip(data_generated, data_ground_truth)):
    sample_w_error_corrections[idx]['Instructions'] = []
    sample_w_error_corrections[idx]['Instances'] = []

    # parsing tool descriptions
    curr_tool_descriptions_parseble = dict()
    tool_set = chain_pred['Function_Description']
    for key, val in tool_set.items():
        all_tools_count += 1
        if val:
            try:
                tool_params = json.loads(val.split('Parameters: ')[1].split('\nOutput:')[0])
                curr_tool_descriptions_parseble[key] = tool_params
            except Exception as e:
                continue
    tool_set['retrieval'] = None
    tool_set['datetime'] = None
    tool_set['getDetails'] = None
    for query_pred, query_gt, steps_pred, steps_gt in zip(chain_pred['Instructions'],
                                                          chain_gt['Instructions'],  
                                                          chain_pred['Instances'],
                                                          chain_gt['Instances']):
        
        assert query_pred == query_gt
        if ('intermediate_steps' in steps_gt and 'intermediate_steps' in steps_pred and 
            len(steps_gt['intermediate_steps']) != 0 and len(steps_pred['intermediate_steps']) != 0):
                try:
                    item_gt, item_pred = steps_gt['intermediate_steps'][0], steps_pred['intermediate_steps'][0]
                    pred_tool_name = item_pred[0][0]
                    pred_tool_params = yaml.load(item_pred[0][1], yaml.Loader)
                    gt_tool_name = item_gt[0][0]
                    gt_tool_params = yaml.load(item_gt[0][1], yaml.Loader)
                    res_pred = check_params(pred_tool_name, pred_tool_params, curr_tool_descriptions_parseble, tool_set)
                    res_gt = check_params(gt_tool_name, gt_tool_params, curr_tool_descriptions_parseble, tool_set)                                
                    if res_gt[0] and not res_pred[0]:
                        sample_w_error_corrections[idx]['Instructions'].append(query_gt)
                        item_gt_copy = deepcopy(steps_gt)
                        item_gt_copy['intermediate_steps'] = [item_pred] + item_gt_copy['intermediate_steps']
                        sample_w_error_corrections[idx]['Instances'].append(item_gt_copy)
                        count_gt_non_pred += 1
                    elif not res_gt[0] and not res_pred[0]:
                        print(pred_tool_name, pred_tool_params, res_pred)
                        print(gt_tool_name, gt_tool_params, res_gt)
                        print('#' * 100)
                        count_non_gt_non_pred += 1
                    elif not res_gt[0] and res_pred[0]:
                        count_non_gt_pred += 1
                    elif res_gt[0] and res_pred[0]:
                        count_gt_pred += 1
                except Exception as e:
                    print(e)
                    error_count += 1
                    print('#' * 100)

count_correction_examples = 0
sample_w_error_corrections_final = []
for item in sample_w_error_corrections:
    if len(item['Instructions']) > 0:
        sample_w_error_corrections_final.append(item)
        count_correction_examples += len(item['Instructions'])

In [ ]:
print(count_non_gt_non_pred)
print(count_non_gt_pred)
print(count_gt_pred)
print(count_gt_non_pred)

Запустим ChatGPT на примерах, которые мы собрали

In [ ]:
from agent.tools import CustomInvalidTool
invalidtool = CustomInvalidTool

In [ ]:
sample_w_error_corrections_final_w_gpt_output = deepcopy(sample_w_error_corrections_final)

count = 0
for item in sample_w_error_corrections_final_w_gpt_output:
    tool_set = load_tools(item)
    tool_set['retrieval'] = None
    tool_set['datetime'] = None
    tool_set['getDetails'] = None
    for chain in item['Instances']:
        name = chain['intermediate_steps'][0][0][0]
        incorrect_params = chain['intermediate_steps'][0][0][1]
        if name in tool_set:
            tool = tool_set[name]
            print(name, incorrect_params)
            print(tool)
            output = tool._run(incorrect_params)
            print(output)
            print('#' * 100)
            count += 1
        else:
            print(name, incorrect_params)
            print(invalidtool)
            output = invalidtool._run(None, name, list(tool_set.keys()))
            print(output)
            print('#' * 100)
            count += 1
        chain['intermediate_steps'][0][-1] = output

In [ ]:
count_correction_examples = 0
for item in sample_w_error_corrections_final_w_gpt_output:
    if len(item['Instructions']) > 0:
        count_correction_examples += len(item['Instructions'])

count_correction_examples

In [ ]:
json.dump(sample_w_error_corrections_final_w_gpt_output, open('data/correction_examples_wrong_params.json', 'w'))

Пересоберу очищенный трейн

In [ ]:
cleaned_train = deepcopy(data_ground_truth)

for idx, chain_gt in enumerate(data_ground_truth):
    cleaned_train[idx]['Instructions'] = []
    cleaned_train[idx]['Instances'] = []

    curr_tool_descriptions_parseble = dict()
    tool_set = chain_gt['Function_Description']
    for key, val in tool_set.items():
        all_tools_count += 1
        if val:
            try:
                tool_params = json.loads(val.split('Parameters: ')[1].split('\nOutput:')[0])
                curr_tool_descriptions_parseble[key] = tool_params
            except Exception as e:
                continue
    tool_set['retrieval'] = None
    tool_set['datetime'] = None
    tool_set['getDetails'] = None

    for query_gt, steps_gt in zip(chain_gt['Instructions'], chain_gt['Instances']):
        if 'intermediate_steps' in steps_gt and len(steps_gt['intermediate_steps']) != 0:
            try:
                flag = True
                for item_gt in steps_gt['intermediate_steps']:
                    gt_tool_name = item_gt[0][0]
                    gt_tool_params = yaml.load(item_gt[0][1], yaml.Loader)
                    res_gt = check_params(gt_tool_name, gt_tool_params, 
                                          curr_tool_descriptions_parseble,
                                          tool_set)
                    if not res_gt[0]:
                        flag = False
                if flag:
                    cleaned_train[idx]['Instructions'].append(query_gt)
                    cleaned_train[idx]['Instances'].append(steps_gt)
            except:
                continue

In [ ]:
num_cleaned_train_examples = 0
for item in cleaned_train:
    num_cleaned_train_examples += len(item['Instances'])
num_cleaned_train_examples

In [ ]:
num_dirty_train_examples

In [ ]:
cleaned_train

In [ ]:
json.dump(cleaned_train,  open('data/cleaned_train_v2_correct.json', 'w'))

Подклею ошибки с галлюцинациями в очищенный трейн

In [ ]:
sample_w_hallucinations = deepcopy(data_ground_truth)
count_hallucinations = 0

for idx, (chain_pred, chain_gt) in enumerate(zip(data_generated, data_ground_truth)):
    sample_w_hallucinations[idx]['Instructions'] = []
    sample_w_hallucinations[idx]['Instances'] = []

    for query_pred, query_gt, steps_pred, steps_gt in zip(chain_pred['Instructions'],
                                                          chain_gt['Instructions'],  
                                                          chain_pred['Instances'],
                                                          chain_gt['Instances']):
        
        if not 'intermediate_steps' in steps_pred and 'intermediate_steps' in steps_gt and len(steps_gt['intermediate_steps']) != 0:
            if 'Could not parse' in steps_pred['error']:
                assistant_thought = steps_pred['error'].split('Could not parse LLM output: `')[1][:-1]
                error = steps_pred['error']
                # concatenate hallucination error with ground truth correct thought
                sample_w_hallucinations[idx]['Instructions'].append(query_gt)
                item_gt_copy = deepcopy(steps_gt)
                item_gt_copy['intermediate_steps'] = [[['hallucination', 'hallucination', assistant_thought], error]] + item_gt_copy['intermediate_steps']
                sample_w_hallucinations[idx]['Instances'].append(item_gt_copy)
                count_hallucinations += 1

sample_w_hallucinations_final = []
for item in sample_w_hallucinations:
    if len(item['Instructions']) > 0:
        sample_w_hallucinations_final.append(item)

In [ ]:
count_hallucinations

In [ ]:
sample_w_hallucinations_final

In [ ]:
final_sample_w_corrections = sample_w_error_corrections_final_w_gpt_output + sample_w_hallucinations_final

In [ ]:
json.dump(final_sample_w_corrections, open('../data/correction_examples_wrong_params_hallucinations.json', 'w'))

Посмотрю на то, как выглядит промпт в обучающей выборке

In [ ]:
from agent.agent_prompts import test_prompt_v1
from agent.custom_agent import CustomZeroShotAgent
from agent.tools import Tool, GetDetailsTool

In [ ]:
tools = [GetDetailsTool()]
prompt = CustomZeroShotAgent.create_prompt(
        tools,
        prefix=test_prompt_v1["prefix"],
        suffix=test_prompt_v1["suffix"],
        format_instructions=test_prompt_v1["format_instructions"],
        input_variables=["input", "agent_scratchpad"]
    )

In [ ]:
prompt

In [ ]:
print(prompt.format(input='test', agent_scratchpad=""))

Загрузим тренировочные данные, чтобы посмотреть на формат

In [ ]:
train_data = json.load(open('data/train_data_correction_examples_wrong_params', 'r'))

In [ ]:
train_data

Разделим train на train и val

In [ ]:
len(data_ground_truth)

In [ ]:
train_part, val_part = data_ground_truth[:268], data_ground_truth[268:]

In [ ]:
json.dump(train_part, open('../data/train_data_train_part.json', 'w'))
json.dump(val_part, open('../data/train_data_val_part.json', 'w'))